In [1]:
import pandas as pd
from pyfaidx import Fasta
import re
import sys

# --- 1. Cấu hình ---
fasta_path = r"D:\Study\5-FA25\AiTa_Lab_Research\Map\Homo_sapiens.GRCh38.dna.primary_assembly.fa"
annotated_variant_file = r"D:\Study\5-FA25\AiTa_Lab_Research\variant_annotated_official_clean.csv"
output_file = r"D:\Study\5-FA25\AiTa_Lab_Research\variant_sequence_positive.csv"

WINDOW = 300
TARGET_LEN = (WINDOW * 2) + 1
PAD_CHAR = 'N'

In [2]:
# --- 2. Các hàm tiện ích ---
def normalize_chrom(chrom):
    """Chuẩn hóa tên chromosome."""
    chrom = str(chrom).strip()
    if chrom.startswith("NC_"):
        num = chrom.split("_")[-1].split(".")[0]
        if num == "000001": return "1"
        elif num == "000002": return "2"
        elif num == "000023": return "X"
        elif num == "000024": return "Y"
        elif num == "000025": return "MT"
        else:
            try:
                num_int = int(num)
                return str(num_int)
            except:
                return chrom
    return chrom.replace("chr", "")

def normalize_chromosome_output(chrom_name):
    """
    Dùng để chuẩn hóa tên chromosome trước khi xuất file CSV.
    Chuyển về dạng UCSC (chr1, chr2...) và loại bỏ contig rác.
    """
    chrom_str = str(chrom_name).strip()

    # Lọc bỏ các contig không xác định (Unplaced/random/alt)
    if 'Un' in chrom_str or 'random' in chrom_str or 'alt' in chrom_str:
        return None # Sẽ trả về None để ta drop hàng này sau

    # Xử lý các trường hợp đặc biệt (MT -> chrM)
    if chrom_str.upper() == 'MT':
        return 'chrM'

    # Nếu nó đã có 'chr' rồi thì thôi
    if chrom_str.startswith('chr'):
        return chrom_str

    # Nếu nó là số (1, 2, X, Y) thì thêm 'chr'
    return 'chr' + chrom_str

def parse_hgvsc_offset(hgvsc_string):
    """Hướng 1: Regex chặt chẽ hơn, yêu cầu dấu +/- phải đứng sau 1 con số"""
    if not isinstance(hgvsc_string, str):
        return None
    
    # Regex giải thích:
    # c\.       : Bắt đầu bằng c.
    # .*?       : Các ký tự ở giữa
    # (?<=\d|\*) : LOOKBEHIND - Ký tự đứng ngay trước dấu +/- PHẢI là số hoặc dấu * (cho 3'UTR)
    # ([+-])    : Nhóm 1 (Dấu)
    # (\d+)     : Nhóm 2 (Giá trị Offset)
    match = re.search(r'c\..*?(?<=\d|\*)([+-])(\d+)', hgvsc_string)
    
    if match:
        sign = match.group(1)
        value = int(match.group(2))
        return -value if sign == '-' else value
    return None

def get_ref_seq(genome, chrom, center_1based, window):
    """Lấy chuỗi DNA 601bp, tự động pad 'N'."""
    target_len = (window * 2) + 1
    start_0based = center_1based - 1 - window
    end_0based = center_1based + window 
    
    try:
        seq = genome[chrom][start_0based:end_0based].upper()
        pad_left = max(0, -start_0based)
        pad_right = max(0, end_0based - len(genome[chrom]))
        final_seq = (PAD_CHAR * pad_left) + seq + (PAD_CHAR * pad_right)
        
        if len(final_seq) != target_len:
             print(f"Lỗi độ dài khi get_ref_seq: {chrom}:{center_1based}. Dự kiến {target_len}, nhận {len(final_seq)}")
             return PAD_CHAR * target_len
        return final_seq
        
    except Exception as e:
        print(f"Lỗi get_ref_seq: {e} tại {chrom}:{center_1based}")
        return PAD_CHAR * target_len

def verify_ref_seq_center(genome, chrom, canonical_center, ref_seq, window):
    """(Hàm kiểm tra) Xác minh base ở tâm `ref_seq` khớp với FASTA."""
    try:
        truth_base = genome[chrom][canonical_center - 1].upper()
        test_base = ref_seq[window].upper()
        if truth_base == test_base:
            return True
        else:
            print(f"--- ⚠️ LỖI CĂN GIỮA! ---")
            print(f"  Tọa độ chuẩn: {chrom}:{canonical_center}")
            print(f"  Base 'Sự thật' từ FASTA: {truth_base}")
            print(f"  Base 'Kiểm tra' tại tâm ref_seq[300]: {test_base}")
            return False
    except Exception as e:
        print(f"LỖI KIỂM TRA (Exception): {e} tại {chrom}:{canonical_center}")
        return False

def normalize_centered_sequence(seq, center_index, target_len, pad_char='N' ):
    """Logic "Cắt/Đệm Đối Xứng" (Symmetric Crop/Pad)."""
    window = target_len // 2
    start = center_index - window
    end = center_index + window + 1
    pad_left = max(0, -start)
    pad_right = max(0, end - len(seq))
    crop_left = max(0, start)
    crop_right = min(len(seq), end)
    
    final_seq = (pad_char * pad_left) + seq[crop_left:crop_right] + (pad_char * pad_right)
    
    if len(final_seq) > target_len:
        over = len(final_seq) - target_len
        final_seq = final_seq[over//2 : -(over - over//2)]
    if len(final_seq) < target_len:
        needed = target_len - len(final_seq)
        final_seq += pad_char * needed
            
    return final_seq

In [3]:
# --- 3. Tải Dữ liệu ---
print(f"🧬 Đang tải FASTA: {fasta_path}")
try:
    genome = Fasta(fasta_path, as_raw=True, sequence_always_upper=True)
except Exception as e:
    print(f"Lỗi nghiêm trọng: Không thể tải file FASTA. Lỗi: {e}")
    sys.exit(1)

print(f"📊 Đang đọc file VEP CSV: {annotated_variant_file}")
try:
    df = pd.read_csv(annotated_variant_file)
except Exception as e:
    print(f"Lỗi nghiêm trọng: Không thể đọc file CSV. Lỗi: {e}")
    sys.exit(1)

# Khởi tạo các cột mới
df['canonical_center'] = None
df['ref_seq'] = None
df['alt_seq'] = None

count_success = 0
count_fail = 0

🧬 Đang tải FASTA: D:\Study\5-FA25\AiTa_Lab_Research\Map\Homo_sapiens.GRCh38.dna.primary_assembly.fa
📊 Đang đọc file VEP CSV: D:\Study\5-FA25\AiTa_Lab_Research\variant_annotated_official_clean.csv


In [4]:
# --- 4. Vòng lặp Xử lý ---
total_rows = len(df)
print(f"⚙️ Bắt đầu xử lý {total_rows} biến thể...")
for index, row in df.iterrows():
    if (index + 1) % 1000 == 0:
        print(f"  ...Đã xử lý {index + 1} / {total_rows}...")

    try:
        # 1. Đọc dữ liệu (nếu đã qua filter)
        chrom = normalize_chrom(row['CHROM'])
        pos = int(row['POS'])
        ref_vcf = str(row['REF']).upper()
        alt_vcf = str(row['ALT']).upper()
        hgvsc_str = str(row['HGVSc'])
        consequence_str = str(row['Consequence'])
        # Lấy nhãn đầu tiên (nghiêm trọng nhất)
        first_consequence = consequence_str.split(',')[0]
        
        # 2. Tìm Tâm Chuẩn (Canonical Center)
        offset = parse_hgvsc_offset(hgvsc_str)
        canonical_center = 0

        if first_consequence == 'splice_donor_variant' and offset is not None:
            canonical_center = pos - (offset - 1)
        elif first_consequence == 'splice_acceptor_variant' and offset is not None:
            canonical_center = pos - (offset + 1)
        else:
            # Missense, UTR, hoặc Splicing bị lỗi offset -> Dùng POS làm tâm
            canonical_center = pos
        
        # 3. Tạo ref_seq
        ref_seq = get_ref_seq(genome, chrom, canonical_center, WINDOW)
        
        if not verify_ref_seq_center(genome, chrom, canonical_center, ref_seq, WINDOW):
            print(f"  CẢNH BÁO: Bỏ qua {chrom}:{pos} do lỗi xác minh căn giữa.")
            count_fail += 1
            continue
            
        # 4. Tạo alt_seq
        # Tính vị trí tương đối của biến thể so với tâm mới
        relative_pos = WINDOW + (pos - canonical_center)
        # Ghép chuỗi ALT
        alt_seq_dynamic = ref_seq[:relative_pos] + alt_vcf + ref_seq[relative_pos + len(ref_vcf):]
        indel_len_change = len(alt_vcf) - len(ref_vcf)
        alt_center_index = WINDOW
        # Nếu biến thể chính là tâm (Missense) -> relative_pos = WINDOW -> alt_center = WINDOW + change
        if relative_pos <= WINDOW:
            alt_center_index = WINDOW + indel_len_change
            
        final_alt_seq = normalize_centered_sequence(
            alt_seq_dynamic, 
            alt_center_index, 
            TARGET_LEN, 
            PAD_CHAR
        )

        # 5. Cập nhật vào DataFrame
        df.at[index, 'canonical_center'] = canonical_center
        df.at[index, 'ref_seq'] = ref_seq
        df.at[index, 'alt_seq'] = final_alt_seq
        
        count_success += 1
        
    except Exception as e:
        print(f"Lỗi xử lý hàng {index}: {e}. Dữ liệu hàng: {row.to_dict()}")

⚙️ Bắt đầu xử lý 3617015 biến thể...
  ...Đã xử lý 1000 / 3617015...
  ...Đã xử lý 2000 / 3617015...
  ...Đã xử lý 3000 / 3617015...
  ...Đã xử lý 4000 / 3617015...
  ...Đã xử lý 5000 / 3617015...
  ...Đã xử lý 6000 / 3617015...
  ...Đã xử lý 7000 / 3617015...
  ...Đã xử lý 8000 / 3617015...
  ...Đã xử lý 9000 / 3617015...
  ...Đã xử lý 10000 / 3617015...
  ...Đã xử lý 11000 / 3617015...
  ...Đã xử lý 12000 / 3617015...
  ...Đã xử lý 13000 / 3617015...
  ...Đã xử lý 14000 / 3617015...
  ...Đã xử lý 15000 / 3617015...
  ...Đã xử lý 16000 / 3617015...
  ...Đã xử lý 17000 / 3617015...
  ...Đã xử lý 18000 / 3617015...
  ...Đã xử lý 19000 / 3617015...
  ...Đã xử lý 20000 / 3617015...
  ...Đã xử lý 21000 / 3617015...
  ...Đã xử lý 22000 / 3617015...
  ...Đã xử lý 23000 / 3617015...
  ...Đã xử lý 24000 / 3617015...
  ...Đã xử lý 25000 / 3617015...
  ...Đã xử lý 26000 / 3617015...
  ...Đã xử lý 27000 / 3617015...
  ...Đã xử lý 28000 / 3617015...
  ...Đã xử lý 29000 / 3617015...
  ...Đã xử lý 3

In [5]:
# --- 5. Hậu xử lý và Lưu file ---
print("\n🧹 Đang làm sạch dữ liệu cuối cùng...")


🧹 Đang làm sạch dữ liệu cuối cùng...


In [6]:
# Check data missing
print('Số missing value sau khi map: \n')
df.isnull().sum()

Số missing value sau khi map: 



AlleleID                    0
Type                        0
Name                        0
GeneID                      0
GeneSymbol                635
HGNC_ID                  4512
ClinicalSignificance        0
ClinSigSimple               0
PhenotypeIDS            51711
PhenotypeList            2652
Origin                      0
OriginSimple                0
Assembly                    0
ChromosomeAccession         0
CHROM                       0
Start                       0
Stop                        0
ReviewStatus                0
VariationID                 0
POS                         0
REF                         0
ALT                         0
Uploaded_variation          0
Consequence                 0
HGVSc                       0
Feature                     0
Protein_id                  0
Protein_position            0
Amino_acids                 3
Codons                      0
canonical_center         3004
ref_seq                  3004
alt_seq                  3004
dtype: int

In [7]:
# 5.1. Loại bỏ những hàng không tạo được sequence (giá trị vẫn là None)
df_final = df.dropna(subset=['ref_seq'])
# 5.2. Chuẩn hóa tên Chromosome output (hàm của bạn)
df_final['CHROM'] = df_final['CHROM'].apply(normalize_chromosome_output)
# 5.3. Loại bỏ các hàng CHROM bị None (do Un/random/alt)
before_clean = len(df_final)
df_final = df_final.dropna(subset=['CHROM'])
after_clean = len(df_final)

print(f" - Tổng input: {total_rows}")
print(f" - Thành công: {count_success}")
print(f" - Thất bại/Lỗi Verify: {count_fail}")
print(f" - Loại bỏ do nhiễm sắc thể rác: {before_clean - after_clean}")
print(f"✅ Tổng số hàng hợp lệ cuối cùng: {len(df_final)}")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11732\4004770591.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['CHROM'] = df_final['CHROM'].apply(normalize_chromosome_output)


 - Tổng input: 3617015
 - Thành công: 3614011
 - Thất bại/Lỗi Verify: 3004
 - Loại bỏ do nhiễm sắc thể rác: 0
✅ Tổng số hàng hợp lệ cuối cùng: 3614011


In [8]:
df_final 

,AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,PhenotypeIDS,PhenotypeList,...,Consequence,HGVSc,Feature,Protein_id,Protein_position,Amino_acids,Codons,canonical_center,ref_seq,alt_seq
0,2193183,single nucleotide variant,NM_001005484.2(OR4F5):c.107A>G (p.Glu36Gly),79501,OR4F5,HGNC:14825,Likely benign,0,MedGen:CN169374,not specified,...,missense_variant,ENST00000641515.2:c.107A>G,ENST00000641515,-,36,E/G,gAa/gGa,69134,GAGACACTGAGAAGCCGAGATAACTGAATTATAAGGCATAGCCAGG...,GAGACACTGAGAAGCCGAGATAACTGAATTATAAGGCATAGCCAGG...
1,4039319,single nucleotide variant,NM_001005484.2(OR4F5):c.281A>G (p.Lys94Arg),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,missense_variant,ENST00000641515.2:c.281A>G,ENST00000641515,-,94,K/R,aAg/aGg,69308,CTGACTTCCTTCTCCTTCTCTTCTTCAAGGTAACTGCAGAGGCTAT...,CTGACTTCCTTCTCCTTCTCTTCTTCAAGGTAACTGCAGAGGCTAT...
2,3374047,single nucleotide variant,NM_001005484.2(OR4F5):c.287T>G (p.Ile96Ser),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,missense_variant,ENST00000641515.2:c.287T>G,ENST00000641515,-,96,I/S,aTt/aGt,69314,TCCTTCTCCTTCTCTTCTTCAAGGTAACTGCAGAGGCTATTTCCTG...,TCCTTCTCCTTCTCTTCTTCAAGGTAACTGCAGAGGCTATTTCCTG...
3,4039320,single nucleotide variant,NM_001005484.2(OR4F5):c.377T>C (p.Met126Thr),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,missense_variant,ENST00000641515.2:c.377T>C,ENST00000641515,-,126,M/T,aTg/aCg,69404,TCATTTTTCTGGGTCTCTCTGATTCTCAGGAACTCCAGACCTTCCT...,TCATTTTTCTGGGTCTCTCTGATTCTCAGGAACTCCAGACCTTCCT...
4,3374048,single nucleotide variant,NM_001005484.2(OR4F5):c.396G>A (p.Met132Ile),79501,OR4F5,HGNC:14825,Uncertain significance,0,MedGen:CN169374,not specified,...,missense_variant,ENST00000641515.2:c.396G>A,ENST00000641515,-,132,M/I,atG/atA,69423,TGATTCTCAGGAACTCCAGACCTTCCTATTTATGTTGTTTTTTGTA...,TGATTCTCAGGAACTCCAGACCTTCCTATTTATGTTGTTTTTTGTA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3617010,706348,single nucleotide variant,NM_002186.3(IL9R):c.991G>A (p.Gly331Arg),3581,IL9R,HGNC:6030,Benign,0,MedGen:C3661900,not provided,...,missense_variant,ENST00000711289.2:c.991G>A,ENST00000711289,-,331,G/R,Ggg/Agg,57196354,CATCTGTACCTCGGTTCAGGCTGCCGTGGGCACCAGGCCCTGTGCT...,CATCTGTACCTCGGTTCAGGCTGCCGTGGGCACCAGGCCCTGTGCT...
3617011,706349,single nucleotide variant,NM_002186.3(IL9R):c.1094G>A (p.Arg365His),3581,IL9R,HGNC:6030,Benign,0,MedGen:C3661900,not provided,...,missense_variant,ENST00000711289.2:c.1094G>A,ENST00000711289,-,365,R/H,cGt/cAt,57196457,ACATGGAGGAAAGACAGAATGTCCAAGACACAGGCGCTGCTTGGCC...,ACATGGAGGAAAGACAGAATGTCCAAGACACAGGCGCTGCTTGGCC...
3617012,4197692,single nucleotide variant,NC_000023.11:g.156022415C>A,-1,NaN,NaN,Likely benign,0,MedGen:C3661900,not provided,...,synonymous_variant,ENST00000711285.2:c.540C>A,ENST00000711285,-,180,I,atC/atA,57208935,GCACCGCCCCCTGGACGAGCGGGCCCTGCAGGTCTGCTGGCTGCGC...,GCACCGCCCCCTGGACGAGCGGGCCCTGCAGGTCTGCTGGCTGCGC...
3617013,3184976,single nucleotide variant,NC_000023.11:g.156023032T>C,-1,NaN,NaN,Likely benign,0,MedGen:C3661900,not provided,...,synonymous_variant,ENST00000711285.2:c.741T>C,ENST00000711285,-,247,Y,taT/taC,57209552,TGGTGCTGTAACAAAGACCCATGTGATGCTGGGGGCAGAGACAGAG...,TGGTGCTGTAACAAAGACCCATGTGATGCTGGGGGCAGAGACAGAG...


In [9]:
# Lưu Kết quả
if not df_final.empty:
    print(f"💾 Đang lưu kết quả vào: {output_file}")
    df_final.to_csv(output_file, index=False)
    print("🎉 Hoàn thành!")
else:
    print("⚠️ Cảnh báo: File kết quả rỗng!")

💾 Đang lưu kết quả vào: D:\Study\5-FA25\AiTa_Lab_Research\variant_sequence_positive.csv
🎉 Hoàn thành!
